# `ceedub` tutorial

In [ ]:
from __future__ import division

import numpy as np
import matplotlib as mpl
mpl.rcParams['image.cmap'] = 'YlOrRd'
import matplotlib.pyplot as plt

import ceedub as cw

%matplotlib inline

### simulate data

In [ ]:
def sine_gaus(ts, f0, tau=1, t0=0, phi=0, amp=1):
    sine = np.sin(2*np.pi * f0 * (ts-t0) + phi)
    gaus = np.exp(-0.5*(ts-t0)**2/tau**2)
    return amp * sine * gaus

In [ ]:
N = 100
dt = 0.10  # sec

ts = np.arange(N)*dt
t0 = ts[N//2]

f0 = 2  # hz
tau = 0.5  # sec

tdat = sine_gaus(ts, f0, tau=tau, t0=t0)

## easy transform

The built in `cwt`, `icwt`, and `cwtfreq` methods use Morlet-Gabor wavelets with width parameter `w0=6`.  This is the basis used by Torrence and Compo (1998).  It is a pretty good all purpose wavelet basis.

There is **no way** to change the basis used by these methods.
To use a different basis you must instantiate a `WaveletBasis` object.

If you plan on doing several transforms using the same data length (`N`) and sampling cadence (`dt`), you should use the `WaveletBasis` object.
These convenience functions create and destroy a `WaveletBasis` each time they are executed, making multiple executions computationally wasteful.

In [ ]:
wdat = cw.cwt(tdat, dt=dt)
tout = cw.icwt(wdat, dt=dt)

fs = cw.wavelet.cwtfreq(N, dt=dt)

wamp = np.abs(wdat)  # wavelet amplitude

### plot transform

In [ ]:
# plotting freq bounds (could go 1/(N*dt) -> 1/(2*dt), but this is fine...)
fmin = 8/(N*dt)
fmax = 1/(2*dt)

# [left, bottom, width, height]
top = [0.1, 0.4, 0.8, 0.6]
bot = [0.1, 0.1, 0.8, 0.3]

fig = plt.figure(figsize=(8,6))

ax1 = fig.add_axes(bot)  # plot time domain data and comparison
ax1.plot(ts, tdat, label='data')
ax1.plot(ts, tout, label='icwt')
ax1.set_xlabel("t (sec)")
ax1.set_ylabel("amplitude")
ax1.legend()

ax2 = fig.add_axes(top)  # plot wavelet domain data
ax2.pcolormesh(ts, fs, wamp, vmin=0)
ax2.set_xticklabels([])
ax2.set_ylim([fmin, fmax])
ax2.set_ylabel("freq (Hz)")
ax2.set_yscale('log')

In theory the `icwt` should perfectly reproduce the input data, but in practice this is not the case.
 * finite sampling effects in both time and frequency
 * roundoff errors in the many FFT-convolutions (forward) and sums (inverse)
 * errors in ICWT normalization from finite integrations

## using a `WaveletBasis`

### default settings

We can initialize a `WaveletBasis` using the defaults and verify that this reproduces the results of the *easy transform*.

In [ ]:
wb = cw.WaveletBasis(N=N, dt=dt)
wdat2 = wb.cwt(tdat)
tout2 = cw.icwt(wdat2, dt=dt)

In [ ]:
np.allclose(wdat, wdat2)

In [ ]:
np.allclose(tout, tout2)

### choosing your own wavelet

In [ ]:
morl3 = cw.MorletWave(w0=3)
morl8 = cw.MorletWave(w0=8)
paul4 = cw.PaulWave(m=4)
paul16 = cw.PaulWave(m=16)

In [ ]:
t_wave = (np.arange(N)-N//2)*dt

fig = plt.figure(figsize=(12,6))
ax1=fig.add_subplot(221)
ax1.plot(t_wave, morl3(t_wave).real, label='Morlet 3 - real')
ax1.plot(t_wave, morl3(t_wave).imag, label='Morlet 3 - imag')
ax1.legend()

ax2=fig.add_subplot(223)
ax2.plot(t_wave, morl8(t_wave).real, label='Morlet 8 - real')
ax2.plot(t_wave, morl8(t_wave).imag, label='Morlet 8 - imag')
ax2.legend()

ax3=fig.add_subplot(222)
ax3.plot(t_wave, paul4(t_wave).real, label='Paul 4 - real')
ax3.plot(t_wave, paul4(t_wave).imag, label='Paul 4 - imag')
ax3.legend()

ax4=fig.add_subplot(224)
ax4.plot(t_wave, paul16(t_wave).real, label='Paul 16 - real')
ax4.plot(t_wave, paul16(t_wave).imag, label='Paul 16 - imag')
ax4.legend()


In [ ]:
mb = cw.WaveletBasis(wavelet=morl3, N=N, dt=dt)
wdat_morl = mb.cwt(tdat)
tout_morl = mb.icwt(wdat_morl)
wamp_morl = np.abs(wdat_morl)

pb = cw.WaveletBasis(wavelet=paul16, N=N, dt=dt)
wdat_paul = pb.cwt(tdat)
tout_paul = mb.icwt(wdat_paul)
wamp_paul = np.abs(wdat_paul)

In [ ]:
# plotting freq bounds (could go 1/(N*dt) -> 1/(2*dt), but this is fine...)
fmin = 8/(N*dt)
fmax = 1/(2*dt)

# four panel bounding boxes!
tl = [0.1, 0.4, 0.4, 0.6]
bl = [0.1, 0.1, 0.4, 0.3]
tr = [0.6, 0.4, 0.4, 0.6]
br = [0.6, 0.1, 0.4, 0.3]

fig = plt.figure(figsize=(16,6))

ax1 = fig.add_axes(bl)  # plot time domain data and comparison
ax1.plot(ts, tdat, label='input data')
ax1.plot(ts, tout_morl, label='morlet icwt')
ax1.set_xlabel("t (sec)")
ax1.set_ylabel("amplitude")
ax1.legend()

ax2 = fig.add_axes(tl)  # plot wavelet domain data
ax2.pcolormesh(ts, fs, wamp_morl, vmin=0)
ax2.set_xticklabels([])
ax2.set_ylim([fmin, fmax])
ax2.set_ylabel("freq (Hz)")
ax2.set_yscale('log')

ax3 = fig.add_axes(br)  # plot time domain data and comparison
ax3.plot(ts, tdat, label='input data')
ax3.plot(ts, tout_paul, label='paul icwt')
ax3.set_xlabel("t (sec)")
ax3.set_ylabel("amplitude")
ax3.legend()

ax4 = fig.add_axes(tr)  # plot wavelet domain data
ax4.pcolormesh(ts, fs, wamp_paul, vmin=0)
ax4.set_xticklabels([])
ax4.set_ylim([fmin, fmax])
ax4.set_ylabel("freq (Hz)")
ax4.set_yscale('log')

## Wavelet Denoising

### simulate noisy data

In [ ]:
N = 100
dt = 0.10  # sec

ts = np.arange(N)*dt
t0 = ts[N//2]

f0 = 2  # hz
tau = 0.5  # sec

signal = sine_gaus(ts, f0, tau=tau, t0=t0, amp=3)
noise = np.random.randn(N)

tdat = signal + noise

In [ ]:
wb = cw.WaveletBasis(N=N, dt=dt)

wdat_orig = wb.cwt(tdat)
wamp_orig = np.abs(wdat_orig)

tout_orig = wb.icwt(wdat_orig)

In [ ]:
minamp = np.sqrt(5)
wdat_DN = wdat_orig.copy()
wdat_DN[wamp_orig<minamp] = 0
wamp_DN = np.abs(wdat_DN)

tout_DN = wb.icwt(wdat_DN)

In [ ]:
# plotting freq bounds (could go 1/(N*dt) -> 1/(2*dt), but this is fine...)
fmin = 8/(N*dt)
fmax = 1/(2*dt)

# four panel bounding boxes!
tl = [0.1, 0.4, 0.4, 0.6]
bl = [0.1, 0.1, 0.4, 0.3]
tr = [0.6, 0.4, 0.4, 0.6]
br = [0.6, 0.1, 0.4, 0.3]

fig = plt.figure(figsize=(16,6))

ax1 = fig.add_axes(bl)  # plot time domain data and comparison
ax1.plot(ts, tdat, label='input data')
ax1.plot(ts, tout_orig, label='icwt')
ax1.set_xlabel("t (sec)")
ax1.set_ylabel("amplitude")
ax1.legend()

ax2 = fig.add_axes(tl)  # plot wavelet domain data
ax2.pcolormesh(ts, fs, wamp_orig, vmin=0)
ax2.set_xticklabels([])
ax2.set_ylim([fmin, fmax])
ax2.set_ylabel("freq (Hz)")
ax2.set_yscale('log')

ax3 = fig.add_axes(br)  # plot time domain data and comparison
ax3.plot(ts, signal, label='input signal')
ax3.plot(ts, tout_DN, label='denoised')
ax3.set_xlabel("t (sec)")
ax3.set_ylabel("amplitude")
ax3.legend()

ax4 = fig.add_axes(tr)  # plot wavelet domain data
ax4.pcolormesh(ts, fs, wamp_DN, vmin=0)
ax4.set_xticklabels([])
ax4.set_ylim([fmin, fmax])
ax4.set_ylabel("freq (Hz)")
ax4.set_yscale('log')